In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_finance_self_service.a_billcount_store_weekly
(
  calmonday DATE,
  store_id STRING,
  billcount INT,
  billcount_mdl INT,
  billcount_fresh INT,
  billcount_mbs INT,
  tpay_trans DOUBLE,
  rev_amt_pos_mbs DOUBLE,
  rev_amt_pos_mld_mbs DOUBLE
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_finance_self_service.a_billcount_store_weekly 
WHERE calmonday >= DATE(DATE_TRUNC('week', DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) - INTERVAL 4 WEEKS);
""")
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_finance_self_service.a_billcount_store_weekly
WITH header AS (
    -- DOANH THU CỦA HỘI VIÊN
    SELECT DISTINCT LTRIM(transaction_no, '0') AS transaction_no 
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_loyalty a 
    WHERE calday >=  DATE(DATE_TRUNC('week', DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) - INTERVAL 4 WEEKS)
),
f_sale_prod AS (
    SELECT
        DATE_TRUNC('WEEK', calday) AS calmonday,
        store_id,
        product_id,
        transaction_type,
        LTRIM(transaction_no, '0') AS transaction_no,
        SUM(sale_amount - tax_amount + CASE WHEN discount_type IN ('ZRTA', 'ZCRT') THEN discount_value ELSE 0 END) AS rev_amt_pos,
        SUM(CASE WHEN payment_method IN ('TQRP', 'ZTPA') THEN tender_value ELSE 0 END) AS tpay_trans
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod
    WHERE calday >=  DATE(DATE_TRUNC('week', DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) - INTERVAL 4 WEEKS)
    GROUP BY 1, 2, 3, 4, 5
),
tpay AS (
  SELECT
    a.calmonday,
    a.store_id,
    SUM(tpay_trans) AS tpay_trans
  FROM f_sale_prod a
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a.store_id = c.store_id
  WHERE c.business_unit = '1500'
    AND c.store_id NOT LIKE '195%'
  GROUP BY 1, 2
),
main AS (
  SELECT
    a.calmonday,
    a.store_id,
    COUNT(DISTINCT CASE WHEN b.bill_count = 1 THEN a.transaction_no END) - COUNT(DISTINCT CASE WHEN b.bill_count = -1 THEN a.transaction_no END) AS billcount,
    COUNT(DISTINCT CASE WHEN b.bill_count = 1 AND product.manufacturer_id IN ('2325', '1915') THEN a.transaction_no END) - COUNT(DISTINCT CASE WHEN b.bill_count = -1 AND product.manufacturer_id IN ('2325', '1915') THEN a.transaction_no END) AS billcount_mdl,
    COUNT(DISTINCT CASE WHEN b.bill_count = 1 AND product.mch2_id = '101' THEN a.transaction_no END) - COUNT(DISTINCT CASE WHEN b.bill_count = -1 AND product.mch2_id = '101' THEN a.transaction_no END) AS billcount_fresh,
    COUNT(DISTINCT CASE WHEN b.bill_count = 1 AND header.transaction_no IS NOT NULL THEN a.transaction_no END) - COUNT(DISTINCT CASE WHEN b.bill_count = -1 AND header.transaction_no IS NOT NULL THEN a.transaction_no END) AS billcount_mbs,
    SUM(CASE WHEN header.transaction_no IS NOT NULL THEN a.rev_amt_pos ELSE 0 END) AS rev_amt_pos_mbs,
    SUM(CASE WHEN header.transaction_no IS NOT NULL AND product.manufacturer_id IN ('2325', '1915') THEN a.rev_amt_pos ELSE 0 END) AS rev_amt_pos_mld_mbs
  FROM f_sale_prod a
  INNER JOIN {catalog_name}.udp_wcm_silver_dim.transaction_type b ON a.transaction_type = b.transaction_type
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product product ON a.product_id = product.product_id
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a.store_id = c.store_id
  LEFT JOIN header ON LTRIM(a.transaction_no, '0') = header.transaction_no
  WHERE c.business_unit = '1500'
    AND c.store_id NOT LIKE '195%'
    AND product.mch2_id IN ('101', '102', '201', '202', '203')
  GROUP BY 1, 2
)
SELECT 
    COALESCE(main.calmonday, tpay.calmonday) AS calmonday,
    COALESCE(main.store_id, tpay.store_id) AS store_id,
    main.billcount,
    main.billcount_mdl,
    main.billcount_fresh,
    main.billcount_mbs,
    tpay.tpay_trans,
    main.rev_amt_pos_mbs,
    main.rev_amt_pos_mld_mbs
FROM main
FULL OUTER JOIN tpay ON main.calmonday = tpay.calmonday AND main.store_id = tpay.store_id
""")